In [1]:
#game.py
import math
import time
import random

class Player():
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        pass

# Takes User Input to return the resulting Value
#This is for the class for Human where it takes the value and converts to integer
class HumanPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-9): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

#Choses a random placement on the board
class RandomComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        square = random.choice(game.available_moves())
        return square

# Uses eitheir a random Value if the board is empty otherwise using the MinMax algorithm to select the best move
class SmartComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())
        else:
            square = self.minimax(game, self.letter)['position']
        return square

#MinMax algorithm going through every single possible move comparing the node values till we reach the optimal value of Mininum or Maximum

    def minimax(self, state, player):
        max_player = self.letter  # yourself
        other_player = 'O' if player == 'X' else 'X'

        # first we want to check if the previous move is a winner
        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (
                        state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -math.inf}  # each score should maximize
        else:
            best = {'position': None, 'score': math.inf}  # each score should minimize
        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)  # simulate a game after making that move

            # undo move
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move  # this represents the move optimal next move

            if player == max_player:  # X is max player
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best

class TicTacToe():
    def __init__(self):
        self.board = self.make_board()
        self.current_winner = None

# Creating the base for the board and the 9 spaces

    @staticmethod
    def make_board():
        return [' ' for _ in range(9)]

#Assigns Value to the board to allow the user wants to select 

    def print_board(self):
        for row in [self.board[i*3:(i+1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # 0 | 1 | 2
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')
# Tells if the space is empty if it wins it assigns to winner

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False
# Checks rows and collums if the letters result is true the same is for the collumns as well

    def winner(self, square, letter):
        # check the row
        row_ind = math.floor(square / 3)
        row = self.board[row_ind*3:(row_ind+1)*3]
        # print('row', row)
        if all([s == letter for s in row]):
            return True
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        # print('col', column)
        if all([s == letter for s in column]):
            return True
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            # print('diag1', diagonal1)
            if all([s == letter for s in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            # print('diag2', diagonal2)
            if all([s == letter for s in diagonal2]):
                return True
        return False
#checks the remaining empty squares and checks avaliable moves

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == " "]

#creates the symbols to play in the game

def play(game, x_player, o_player, print_game=True):

    if print_game:
        game.print_board_nums()

    letter = 'X'
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)
        if game.make_move(square, letter):

            if print_game:
                print(letter + ' makes a move to square {}'.format(square))
                game.print_board()
                print('')

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter  # ends the loop and exits the game
            letter = 'O' if letter == 'X' else 'X'  # switches player

        time.sleep(.8)

    if print_game:
        print('It\'s a tie!')

In [8]:
#ai_apuente2016.py functions and data

from game import *

DESTINATION='BUCHAREST'
city_paths = {
    'ARAD':             [['ZERIND', 75], ['TIMISOARA', 118], ['SIBIU', 140]],
	'BUCHAREST':        [['URZICENI', 85], ['GIURGIU', 90], ['PITESTI', 101], ['FAGARAS', 211]],
	'CRAIOVA':          [['DOBRETA', 120], ['PITESTI', 138], ['RIMNICU VILCEA', 146]],
	'DOBRETA':          [['MEHADIA', 75], ['CRAIOVA', 120]],
	'EFORIE':           [['HIRSOVA', 86]],
	'FAGARAS':          [['SIBIU', 99], ['BUCHAREST', 211]],
	'GIURGIU':          [['BUCHAREST', 90]],
	'HIRSOVA':          [['EFORIE', 86], ['URZICENI', 98]],
	'IASI':             [['NEAMT', 87], ['VASLUI', 92]],
	'LUGOJ':            [['MEHADIA', 70], ['TIMISOARA', 111]],
	'MEHADIA':          [['LUGOJ', 70], ['DOBRETA', 75]],
	'NEAMT':            [['IASI', 87]],
	'ORADEA':           [['ZERIND', 71], ['SIBIU', 151]],
	'PITESTI':          [['RIMNICU VILCEA', 97], ['BUCHAREST', 101], ['CRAIOVA', 138]],
	'RIMNICU VILCEA':   [['SIBIU', 80], ['PITESTI', 97], ['CRAIOVA', 146]],
	'SIBIU':            [['RIMNICU VILCEA', 80], ['FAGARAS', 99], ['ARAD', 140], ['ORADEA', 151]],
	'TIMISOARA':        [['LUGOJ', 111], ['ARAD', 118]],
	'URZICENI':         [['BUCHAREST', 85], ['HIRSOVA', 98]],
	'VASLUI':           [['IASI', 92], ['URZICENI', 142]],
	'ZERIND':           [['ORADEA', 71], ['ARAD', 75]]
}
line_to_Bucharest={
    'ARAD':             418,
	'BUCHAREST':        0,
	'CRAIOVA':          239,
	'DOBRETA':          359,
	'EFORIE':           269,
	'FAGARAS':          211,
	'GIURGIU':          90,
	'HIRSOVA':          183,
	'IASI':             319,
	'LUGOJ':            504,
	'MEHADIA':          434,
	'NEAMT':            406,
	'ORADEA':           429,
	'PITESTI':          101,
	'RIMNICU VILCEA':   198,
	'SIBIU':            278,
	'TIMISOARA':        536,
	'URZICENI':         85,
	'VASLUI':           227,
	'ZERIND':           493
}

def BFS(city_paths:dict, start:str, goal:str) -> list:
	"""
    Accepts a start city and a goal city on city_paths and uses Binary First Search.
    Returns a list containing path of cities traveled through in search.
    """
	explored = []
	
	# Queue for traversing the
	# graph in the BFS
	queue = [[start]]
	
	# If the desired node is
	# reached
	if start == goal:
		return [start]
	
	# Loop to traverse the graph
	# with the help of the queue
	while queue:
		path = queue.pop(0)
		node = path[-1]
		
		# Condition to check if the
		# current node is not visited
		if node not in explored:
			neighbours = city_paths[node]
			
			# Loop to iterate over the
			# neighbours of the node
			for neighbour_node in neighbours:
				neighbour = neighbour_node[0]
				new_path = list(path)
				new_path.append(neighbour)
				queue.append(new_path)
				
				# Condition to check if the
				# neighbour node is the goal
				if neighbour == goal:
					#print("BFS Path = ", *new_path)
					return new_path
			explored.append(node)

	# Condition when the nodes
	# are not connected
	print("So sorry, but a connecting"\
				" path doesn't exist :(")
	return None

def DFS(city_paths:dict, start:str, goal:str) -> list:
    """
    Accepts a start city and a goal city on city_paths and uses Depth First Search.
    Returns a list containing path of cities traveled through in search.
    """
    explored = []

    # Stack for traversing the
    # graph in the DFS
    stack = [[start]]

    # If the desired node is
    # reached
    if start == goal:
        return [start]

    # Loop to traverse the graph
    # with the help of the stack
    while stack:
        path = stack.pop()
        node = path[-1]
        
        # Condition to check if the
        # current node is not visited
        if node not in explored:
            neighbours = city_paths[node]
            
            # Loop to iterate over the
            # neighbours of the node
            for neighbour_node in neighbours:
                neighbour = neighbour_node[0]
                new_path = list(path)
                new_path.append(neighbour)
                stack.append(new_path)
                
                # Condition to check if the
                # neighbour node is the goal
                if neighbour == goal:
                    #print("DFS Path = ", *new_path)
                    return new_path
            explored.append(node)

    # Condition when the nodes
    # are not connected
    print("So sorry, but a connecting"\
                " path doesn't exist :(")
    return None

def trip_distance(city_history:list) -> None:
    """
    Prints city-to-city travel distance as well as total distance traveled.
    """
    total=0
    itinerary=city_history
    for city in enumerate(itinerary):#city==(index,city name)
        if itinerary[city[0]]==itinerary[-1]:
            break
        for path in city_paths[city[1]]:
            if path[0] in itinerary[city[0]+1]:
                print(itinerary[city[0]],
                    "->", itinerary[city[0]+1],
                    "=",path[1],"km")
                total+=path[1]
    print("The total distance traveled will be",total,"km.")
    return None
####################### CLASS ####################################
class Traverse:

    distance_traveled=0
    city_history=[]

    def __init__(self,city:str,root:bool) -> None:
        """
        Creates a Traverse object adding city to city_history.\n
        The root variable indicates whether object created is root node
        or object formed through aStar recursion.
        """
        self.distance_traveled=0
        self.city_history.append(city)
        if root:
            self.city_history.clear()
            self.city_history.append(city)
            self.distance_traveled=0
    
    def aStar(self) -> tuple:
        """
        Recursively calls itself after finding the lowest cost path
        at each city.\n
        Returns (distance_traveled, city_history).
        """
        if self.city_history[-1]==DESTINATION: return self;
        paths=[]
        costs=[]
        for i in range(len(city_paths[self.city_history[-1]])):
            if city_paths[self.city_history[-1]][i][0] not in self.city_history:    #checks that city in city_paths has not been visited
                paths.append(city_paths[self.city_history[-1]][i])
        for path in paths:#path=[city,distance]
            costs.append(self.calculate_cost(path))
        lowest_cost_path=paths[costs.index(min(costs))]
        nextCity=Traverse(lowest_cost_path[0],False)
        self.distance_traveled+=lowest_cost_path[1]
        return nextCity.aStar()

    def calculate_cost(self,path:list) -> int:
        """f(n)=g(n)+h(n)"""
        return (self.distance_traveled+path[1])+line_to_Bucharest[path[0]]
####################    PROGRAMS    #####################
def find_shortest_route() -> None:
    """
    Loops path-finding prompt.
    """
    while True:
        city=input(str(list(city_paths.keys()))+" Which city do you want to depart from? ").upper()
        while city not in list(city_paths.keys()):
            city=input(str(list(city_paths.keys()))+" Incorrect. Please select a city from the list: " ).upper()
        algo=input("A -> A* | B -> BFS | D -> DFS. Which algorithm would you like to use? ").upper()
        while algo not in ['A','B','D']:
            algo=input("A -> A* | B -> BFS | D -> DFS. Incorrect. Please select an algorithm from the list: " ).upper()
        if algo == 'A':
            print("\nYou have chosen to travel from",city,
                "to",DESTINATION,"using the A* heuristic search.")
            traverser=Traverse(city,True)
            traverser.aStar()
            trip_distance(traverser.city_history)
        elif algo == 'B':
            print("\nYou have chosen to travel from",city,
                "to",DESTINATION,"using the Breadth First Search.")
            traverser=BFS(city_paths,city,DESTINATION)
            trip_distance(traverser)
        else:
            print("\nYou have chosen to travel from",city,
                "to",DESTINATION,"using the Depth First Search.")
            traverser=DFS(city_paths,city,DESTINATION)
            trip_distance(traverser)
        if input("Type anything to find another route or X to exit: ").upper() == 'X':
            break
    return None

def game() -> None:
    while True:
        selected=input("Would you like to start first? Y for Yes, N for No: ").upper()
        while selected not in ['Y','N']:
            selected=input("Incorrect. Would you like to start first? Y for Yes, N for No: ").upper()
        if selected=='N':
            x_player = SmartComputerPlayer('X')
            o_player = HumanPlayer('O')
        else:
            x_player = HumanPlayer('X')
            o_player = SmartComputerPlayer('O')
        t = TicTacToe()
        play(t, x_player, o_player, print_game=True)
        if input("Type anything to play another game or X to exit: ").upper() == 'X':
            break
    return None

In [9]:
#driver
# CAP 4630 Intro to AI
# Assignment 1
# 06/11/2021
# Adrian Puente, Meike Buettner, Mohammad Khan

while True:
    answer=input("Enter 'Path' to find a route to Bucharest, 'Game' to play Tic-Tac-Toe, or X to exit: ").upper()
    while answer not in ['PATH','GAME','X']:
            answer=input("Incorrect. Please select a program to run: " ).upper()
    if answer == 'PATH':
        print(
        """ 
        This app finds a route from a city to the city of Bucharest.
        The user provides starting city and routing algorithm as input.
        The output is the algorithm chosen, cities traveled through,
        individual distances traveled, and total distance traveled.
        """)
        find_shortest_route()
    elif answer == 'GAME':
        game()
    else:
        break

 
        This app finds a route from a city to the city of Bucharest.
        The user provides starting city and routing algorithm as input.
        The output is the algorithm chosen, cities traveled through,
        individual distances traveled, and total distance traveled.
        

You have chosen to travel from ARAD to BUCHAREST using the Breadth First Search.
ARAD -> SIBIU = 140 km
SIBIU -> FAGARAS = 99 km
FAGARAS -> BUCHAREST = 211 km
The total distance traveled will be 450 km.
